In [608]:
using DataFrames
using CSV
using Random

df = DataFrame(CSV.File("iris.data", header=false))
iris_names = ["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
rename!(df, iris_names)

,sepal_len,sepal_wid,petal_len,petal_wid,class
,Float64,Float64,Float64,Float64,String15
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa


In [609]:
function train_test_spliter(data, train_Percent = 2/3)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, train_Percent *n))
    test_idx = view(idx, (floor(Int, train_Percent *n)+1):n)
    data[train_idx,:], data[test_idx,:]
end


train_test_spliter (generic function with 2 methods)

In [610]:
function cal_total_entropy(data)
    class_list = unique(data[:,end])
    n = size(data,1)
    total_entropy = 0
    for class in class_list
        p = count(==(class),data[:,end]) / n
        total_entropy = total_entropy + (-p*log2(p))
    end
    return total_entropy
end

cal_total_entropy (generic function with 1 method)

In [611]:
function cal_entropy(feature_data)
    #=
    feature_data has 2 col:
        1- col values that sastify certain condition
        2- result col (last col)
    =#
    n = size(feature_data,1)
    entropy = 0

    unq = unique(feature_data[:,end])
    for class in unq
        entropy_class = 0
        p = count(==(class), feature_data[:,end]) / n
        entropy_class = -p*log2(p)
        entropy += entropy_class
    end
    return entropy
end


cal_entropy (generic function with 1 method)

In [612]:
function cal_feature_info(data , n)
    #=
    data: datafame (upper/lower)
    n: #row before split

    return pob * entropy
    =#
    counter = size(data,1)
    prob = counter / n
    entropy = cal_entropy(data)    

    return prob*entropy
end

cal_feature_info (generic function with 1 method)

In [613]:
function cal_infomation_gain(feature_name, data)
    #= 
    feature_name = name of feature
    data = whole dataframe
    =#
    feature_values = unique(data[:,feature_name])
    n = size(data,1)
    min_feature_info = 9999.0
    cutoff = 0.0
    for value in feature_values 
        feature_data_upper = filter(n->n[feature_name] >= value,data)
        feature_data_lower = filter(n->n[feature_name] < value,data)

        feature_info_upper = cal_feature_info(feature_data_upper,n)
        feature_info_lower = cal_feature_info(feature_data_lower,n)
        feature_info = feature_info_upper + feature_info_lower
        if feature_info < min_feature_info
            cutoff = value
            min_feature_info = feature_info
        end
    end
    
    return cal_entropy(data) - min_feature_info, cutoff
    
end


cal_infomation_gain (generic function with 1 method)

In [614]:

function find_most_infomative_feature(data)

    feature_list = names(data)[1:end-1]

    max_inf_gain = -1
    max_feature = Nothing
    feature_cutoff = 0.0
    for feature in feature_list
        feature_info_gain, cutoff = cal_infomation_gain(feature,data)
        if max_inf_gain < feature_info_gain
            max_inf_gain = feature_info_gain
            max_feature = feature
            feature_cutoff = cutoff
        end
    end

    return max_feature, feature_cutoff
end


find_most_infomative_feature (generic function with 1 method)

In [615]:
function generate_sub_tree(feature_name,cutoff, data)
    #=
    feature_name =  name of the feature that we want to add to tree and shrink dataset
    cutoff: cutoff value of feature
    data: DataFrame
    =#
    #feature_value_count_dic = countmap(data[:,feature_name])
    tree =Dict{Any,Any}()
    upper = filter(n->n[feature_name] >= cutoff,data)
    lower = filter(n->n[feature_name] <  cutoff,data)

    #upper
    if size(upper,1) != 0
        count = size(upper,1)
        assigned_to_node = false
        for c in unique(data[:,end])
            class_count = size(filter(n->n[end] == c, upper),1)
            if class_count == count
                push!(tree, (" >= " * string(cutoff)) => c)
                data = filter(n->n[feature_name] < cutoff,data)
                assigned_to_node = true
            end
        end
        if assigned_to_node == false
            push!(tree, (" >= "*string(cutoff)) => "?")
        end
    end
    #lower
    if size(lower,1) != 0

        count = size(lower,1)
        assigned_to_node = false
        for c in unique(data[:,end])
            class_count = size(filter(n->n[end] == c, lower),1)
            if class_count == count
                push!(tree, (" < " * string(cutoff)) => c)
                data = filter(n->n[feature_name] >= cutoff,data)
                assigned_to_node = true
            end
        end
        if assigned_to_node == false
            push!(tree, (" < "*string(cutoff)) => "?")
        end
    end
    return tree, data
    
end

generate_sub_tree (generic function with 1 method)

In [616]:
function make_tree(root, prev_feature_value, data)
    if size(data,1) != 0
        max_info_feature,cutoff = find_most_infomative_feature(data)
        tree ,data = generate_sub_tree(max_info_feature, cutoff, data)
        next_root = nothing

        if prev_feature_value != nothing
            root[prev_feature_value] = Dict{Any,Any}()
            #root[prev_feature_value][max_info_feature] = tree
            push!(root,prev_feature_value=>Dict{Any,Any}(max_info_feature => tree))
            next_root = root[prev_feature_value][max_info_feature]

        else
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        end

        for (node,branch) in next_root
            #todo
            temp_node = node
            node = lstrip(node)
            node = split(node," ")

            compare_symbol = string(node[1])
            value = parse(Float64,string(node[2]))
            if branch == "?"
                if compare_symbol == ">="
                feature_value_data = filter(n->n[max_info_feature] >= value, data)
                else 
                feature_value_data = filter(n->n[max_info_feature] < value, data)
                end
                make_tree(next_root, temp_node, feature_value_data)
            end
        end
    end
end




make_tree (generic function with 1 method)

In [617]:
function id3(data)
    train_data = data
    tree = Dict{Any,Any}()
    make_tree(tree, nothing, train_data)

    return tree
    
end

id3 (generic function with 1 method)

In [618]:
function predict(tree, instance)
    if typeof(tree) != Dict{Any,Any}
        return tree
    else
        root_node = pop!(string.(keys(tree)))
        feature_value = instance[root_node]


        value = string.(keys(tree[root_node]))[1]
        value = parse(Float64,split(value," ")[end])

        if feature_value >= value
            feature_value_string = string(" >= ",string(value))
            return predict(tree[root_node][feature_value_string], instance)
        else
            feature_value_string = string(" < ",string(value))
            return predict(tree[root_node][feature_value_string], instance)
        end
    
    end   
end

predict (generic function with 1 method)

In [619]:
function cal_accuracy(test_data, tree)
    correct = 0
    n = nrow(test_data)
    for i in eachrow(test_data)
        result = predict(tree,i)
        if result == i[end]
            correct += 1
        end
    end

    return correct / n * 100
    
end

cal_accuracy (generic function with 1 method)

In [620]:
for i in 1:50
    print(i," ")
    Random.seed!(i)
train, test =train_test_spliter(df)
tree = id3(train)

println(string(cal_accuracy(test,tree)),"%")
end

1 96.0%
2 98.0%
3 96.0%
4 90.0%
5 96.0%
6 98.0%
7 90.0%
8 94.0%
9 92.0%
10 92.0%
11 96.0%
12 92.0%
13 94.0%
14 92.0%
15 94.0%
16 94.0%
17 94.0%
18 98.0%
19 94.0%
20 94.0%
21 100.0%
22 96.0%
23 94.0%
24 96.0%
25 94.0%
26 94.0%
27 94.0%
28 94.0%
29 90.0%
30 96.0%
31 90.0%
32 94.0%
33 92.0%
34 86.0%
35 94.0%
36 88.0%
37 92.0%
38 94.0%
39 92.0%
40 92.0%
41 94.0%
42 90.0%
43 94.0%
44 96.0%
45 94.0%
46 98.0%
47 88.0%
48 90.0%
49 94.0%
50 92.0%
